In [2]:
import glob
from utils.reference_fronts import ReferenceFronts
from utils.file_utils import parse_exp_log_dir, load_n_evaluation_log, parse_meta
from utils.performance_evaluator import PerformanceEvaluator

exp_dir_pattern = "../data/N-obj*"
for directory in glob.glob(exp_dir_pattern):
    meta_data = parse_meta(exp_dir=directory)

ValueError: Invalid experiment experiment path